In [16]:
from pandas import pandas as pd, DataFrame
from torch import torch, Tensor
import numpy as np

In [17]:
def relu(tensor: Tensor):
    return torch.clip(tensor, 0.)

# Import Competition Data

In [18]:
training_data = pd.read_csv("./downloads/train.csv")

In [19]:
training_data = training_data.dropna()
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [20]:
train_df=training_data.sample(frac=0.8,random_state=200)
test_df=training_data.drop(train_df.index)

In [21]:
def get_x_values(df: DataFrame):
    x_values = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'])

    x_values['is_male'] = (df["Sex"] == "male").astype(int)
    x_values['embarked_c'] = (df["Embarked"] == "C").astype(int)
    x_values['embarked_s'] = (df["Embarked"] == "S").astype(int)
    x_values['pclass_1'] = (df["Pclass"] == 1).astype(int)
    x_values['pclass_2'] = (df["Pclass"] == 2).astype(int)
    
    x_values = x_values.drop(columns=['Sex', 'Embarked', 'Pclass'])

    # Normalize numeric columns
    x_values['Age'] = x_values['Age'] / x_values['Age'].max()
    # x_values['Fare'] = np.log10(x_values['Fare']+1)
    x_values['Fare'] = x_values['Fare'] / x_values['Fare'].max()
    x_values['SibSp'] = x_values['SibSp'] / x_values['SibSp'].max()
    x_values['Parch'] = x_values['Parch'] / x_values['Parch'].max()
    
    # add bias
    x_values['ones'] = 1
    
    return x_values

In [22]:
def get_y_values(df: DataFrame):
    return df["Survived"]

In [23]:
train_x = get_x_values(train_df)
test_x = get_x_values(test_df)

train_y = get_y_values(train_df)
test_y = get_y_values(test_df)


In [24]:
m_params_1 = torch.randn(train_x.shape[1], requires_grad=True)
m_params_2 = torch.randn(train_x.shape[1], requires_grad=True)

In [25]:
m_params_1

tensor([-1.1539, -0.3985,  1.2789, -1.3050,  0.0401,  0.1261, -1.2428, -0.1117,
        -0.1956,  0.7959], requires_grad=True)

In [26]:
# make prediction using a double ReLu
def f(x: DataFrame):
    # Convert DataFrame to tensor
    x_tensor = torch.tensor(x.values, dtype=torch.float32)
    
    linear_1 = x_tensor @ m_params_1
    linear_2 = x_tensor @ m_params_2

    return relu(linear_1) + relu(linear_2)

In [27]:
def train(epochs: int, learning_rate = 1e-3):
    global m_params_1, m_params_2
    
    # Calculate the step intervals
    step_size = epochs // 4  # This will give us 5 points (0, 1/4, 2/4, 3/4, 4/4)
    print_steps = [0, step_size, step_size*2, step_size*3, epochs-1]
    
    for i in range(epochs):
        # Zero out gradients
        if m_params_1.grad is not None:
            m_params_1.grad.zero_()
        if m_params_2.grad is not None:
            m_params_2.grad.zero_()
            
        # Forward pass
        predictions = f(train_x)
        y_tensor = torch.tensor(train_y.values, dtype=torch.float32)
        loss = ((predictions - y_tensor) ** 2).mean()
        
        # Backward pass
        loss.backward()
        
        with torch.no_grad():
            m_params_1 -= m_params_1.grad * learning_rate
            m_params_1.grad = None
            m_params_2 -= m_params_2.grad * learning_rate
            m_params_2.grad = None
            
        # Print only at specific steps
        if i in print_steps:
            print(f'step={i}; loss={loss:.2f}')

In [28]:
print('Print weights before training')
pd.DataFrame({
    'Parameter': train_x.columns,
    'm_params_1': [f"{x:.2f}" for x in m_params_1.tolist()],
    'm_params_2': [f"{x:.2f}" for x in m_params_2.tolist()]
})

Print weights before training


,Parameter,m_params_1,m_params_2
0,Age,-1.15,-0.09
1,SibSp,-0.40,-0.40
2,Parch,1.28,-0.01
3,Fare,-1.31,1.26
4,is_male,0.04,-0.26
5,embarked_c,0.13,0.39
6,embarked_s,-1.24,0.91
7,pclass_1,-0.11,-1.22
8,pclass_2,-0.20,0.94
9,ones,0.80,-0.35


In [29]:
train(10000, 1e-2)

step=0; loss=0.54
step=2500; loss=0.17
step=5000; loss=0.16
step=7500; loss=0.16
step=9999; loss=0.16


In [30]:
# Get predictions and convert to binary (0 or 1)
train_preds = f(test_x)
binary_preds = (train_preds.sigmoid() > 0.5).float()  # Convert to 0 or 1 based on threshold of 0.5

# Calculate accuracy
correct = (binary_preds == torch.tensor(test_y.values, dtype=torch.float32)).sum().item()
total = len(test_y)
accuracy = correct / total

print(f'Accuracy: {accuracy:.2%} ({correct}/{total} correct)')
pd.DataFrame({
    'Parameter': train_x.columns,
    'm_params_1': [f"{x:.2f}" for x in m_params_1.tolist()],
    'm_params_2': [f"{x:.2f}" for x in m_params_2.tolist()]
})

Accuracy: 83.78% (31/37 correct)


,Parameter,m_params_1,m_params_2
0,Age,-0.39,-0.56
1,SibSp,0.32,-0.31
2,Parch,0.12,-0.30
3,Fare,-0.93,1.25
4,is_male,-0.57,0.33
5,embarked_c,0.01,0.36
6,embarked_s,-0.07,0.68
7,pclass_1,0.32,-1.22
8,pclass_2,-0.13,0.42
9,ones,0.95,-0.61
